In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import importlib
import time
import datetime
import json
import threading

import shioaji as sj
import login.shioaji_login as shioaji_login
from shioaji.constant import Action, OptionRight, StockPriceType, FuturesOrderType, FuturesOCType #選擇權下單，多匯入一個OptionRight常數
importlib.reload(shioaji_login)

import tools.globals as globals
import tools.get_snap_options as snap

In [2]:
def update_config():
    """
    Threading function.
    每秒更新config相關參數
    
    :global param: get_strike_time
    :return: None
    """

    pre_get_strike_time = None
    
    while(True):

        with open('config.json') as f:
            config_data = json.load(f)

            globals.get_strike_time =  datetime.datetime.strptime(config_data['get_strike_time'], '%H:%M:%S').time()

            if(pre_get_strike_time != globals.get_strike_time):
                print(f'Get strike time has been set to {globals.get_strike_time}')
                pre_get_strike_time = globals.get_strike_time

            time.sleep(1)



In [3]:
def update_snap_options():
    """
    Threading function.
    每秒偵測,若現在時間符合get_strike_time, 則找尋當下價平 
    
    :global param: get_strike_time
    :return: None
    """

    while(True): 
        now = datetime.datetime.now()
        
        if(now.time().replace(microsecond=0) == globals.get_strike_time):
            snap.get_snap_options()
            snap.get_at_the_money_info()
            print("現在價平和之檔位及成交點數和: ", globals.at_the_money_code, globals.at_the_money)

        time.sleep(1)

In [4]:
def place_cb(stat, msg):
    """
    Called every time an order or a deal has been detected.
    
    :global param: msg_list ()

    return: none
    """
    print('my_place_callback')

In [5]:
def main():
    """
    Controlled whole program, make app execute.
    
    :global param: api
    """
    # initialize all global variables
    globals.initialize()

    # log in
    globals.api = shioaji_login.login()
    globals.api.set_order_callback(place_cb)

    # Start update config thread. All config variable will be updated every second.
    update_config_thread = threading.Thread(target = update_config)
    update_config_thread.start()
    time.sleep(1)

    update_snap_options_thread = threading.Thread(target = update_snap_options)
    update_snap_options_thread.start()
    time.sleep(1)

    snap.get_snap_options()
    snap.get_at_the_money_info()
    print("現在價平和之檔位及成交點數和: ", globals.at_the_money_code, globals.at_the_money)
    
    # print(api.Contracts.Options.TX4)



In [6]:
if __name__ == "__main__":
    main() 

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
Login with ID R124743829
Login status: [FutureAccount(person_id='R124743829', broker_id='F002000', account_id='1697256', signed=True, username='劉庭銘'), Account(account_type=<AccountType.H: 'H'>, person_id='R124743829', broker_id='9A9X', account_id='0150198', username='劉庭銘\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000'), StockAccount(person_id='R124743829', broker_id='9A9X', account_id='0510980', signed=True, username='劉庭銘\u3000\u3000')]
Activating CA at the path C:/sinopac/Sinopac.pfx


Get strike time has been set to 21:05:00
現在價平和之檔位及成交點數和:  TX514600 155.0


In [18]:
option_code = globals.at_the_money_code +  snap.get_option_code('C') # 先預設下call
print(option_code)
contract = globals.api.Contracts.Options[option_code] #選擇權Contract
order = globals.api.Order(action=Action.Buy,
    price=6, #價格
    quantity=1, #口數
    price_type=StockPriceType.LMT,
    order_type=FuturesOrderType.ROD,
    octype=FuturesOCType.Auto, #倉別，使用自動
    OptionRight=OptionRight.Call, #選擇權類型
    account=globals.api.futopt_account #下單帳戶指定期貨帳戶
)
globals.api.place_order(contract, order)

TX514600K2


Trade(contract=Option(code='TX514600K2', symbol='TX520221114600C', name='臺指選擇權11W5月 14600C', category='TX5', delivery_month='202211', delivery_date='2022/11/30', strike_price=14600, option_right=<OptionRight.Call: 'C'>, underlying_kind='I', unit=1, limit_up=1680.0, limit_down=0.1, reference=204.0, update_date='2022/11/28'), order=Order(action=<Action.Buy: 'Buy'>, price=6, quantity=1, account=Account(account_type=<AccountType.Future: 'F'>, person_id='R124743829', broker_id='F002000', account_id='1697256', signed=True), price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>), status=OrderStatus(status=<Status.Failed: 'Failed'>, status_code='74008', order_datetime=datetime.datetime(2022, 11, 28, 21, 52, 15, 808734), msg='憑證驗章失敗,請至3310憑證管理畫面憑證查詢狀態', deals=[]))